In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Cargar los archivos CSV
df_fbref = pd.read_csv('./final_data/fbref.csv')
df_xvalue = pd.read_csv('./final_data/xvalue.csv')

In [46]:
print(df_fbref.shape)
print(df_xvalue.shape)

(33475, 14)
(27015, 34)


In [47]:
print("Columnas de df_fbref:", df_fbref.columns)
print("Columnas de df_xvalue:", df_xvalue.columns)

Columnas de df_fbref: Index(['datetime', 'competition', 'goals_scored', 'goals_conceded', 'opponent',
       'possession', 'formation', 'shots', 'shots_on_target', 'penaltys_made',
       'penaltys_attempted', 'season', 'team', 'penalty_shootout'],
      dtype='object')
Columnas de df_xvalue: Index(['team', 'opponent', 'side', 'competition', 'year', 'goals_scored',
       'expected_goals', 'expected_goals_in_open_play',
       'expected_goals_from_counters', 'expected_goals_from_set_pieces',
       'shot_accuracy', 'goals_conceded', 'expected_goals_conceded',
       'expected_goals_conceded_in_open_play',
       'expected_goals_conceded_from_counters',
       'expected_goals_conceded_from_set_pieces', 'total_shots_conceded',
       'shots_on_target_conceded', 'defensive_event_height', 'field_tilt',
       'expected_threat_from_passes', 'total_passes', 'dribbles',
       'touches_opponent_box', 'through_balls', 'crosses', 'one_twos', 'ppda',
       'gpi', 'total_fouls_won', 'total_fouls

In [48]:
df_fbref['datetime'] = pd.to_datetime(df_fbref['datetime'], format='%d/%m/%Y %H:%M')

In [49]:
def determine_fbref_season(date, competition):
    # Diccionarios con los rangos de fechas para cada temporada y competición
    seasons_dict = {
        'Liga Profesional Argentina': [
            ((datetime(2017, 8, 20), datetime(2018, 5, 20)), '2017/2018'),
            ((datetime(2018, 8, 5), datetime(2019, 4, 13)), '2018/2019'),
            ((datetime(2019, 7, 21), datetime(2020, 3, 14)), '2019/2020'),
            ((datetime(2021, 7, 9), datetime(2021, 12, 18)), '2021'),
            ((datetime(2022, 6, 1), datetime(2022, 10, 30)), '2022')
        ],
        'Brazilian Serie A': [
            ((datetime(2017, 5, 6), datetime(2017, 12, 10)), '2017'),
            ((datetime(2018, 4, 7), datetime(2018, 12, 10)), '2018'),
            ((datetime(2019, 4, 20), datetime(2019, 12, 15)), '2019'),
            ((datetime(2020, 8, 2), datetime(2021, 3, 1)), '2020'),
            ((datetime(2021, 5, 23), datetime(2021, 12, 16)), '2021'),
            ((datetime(2022, 4, 1), datetime(2022, 11, 20)), '2022')
        ],
         'Premier League': [
            ((datetime(2017, 8, 5), datetime(2018, 5, 25)), '2017/2018'),
            ((datetime(2018, 8, 12), datetime(2019, 5, 27)), '2018/2019'),
            ((datetime(2019, 8, 10), datetime(2020, 7, 30)), '2019/2020'),
            ((datetime(2020, 9, 13), datetime(2021, 5, 30)), '2020/2021'),
            ((datetime(2021, 8, 7), datetime(2022, 5, 30)), '2021/2022'),
            ((datetime(2022, 8, 1), datetime(2023, 6, 20)), '2022/2023')
        ],
        'Ligue 1': [
            ((datetime(2017, 8, 5), datetime(2018, 5, 27)), '2017/2018'),
            ((datetime(2018, 8, 5), datetime(2019, 5, 27)), '2018/2019'),
            ((datetime(2019, 8, 3), datetime(2020, 7, 30)), '2019/2020'),
            ((datetime(2020, 9, 9), datetime(2021, 5, 30)), '2020/2021'),
            ((datetime(2021, 8, 5), datetime(2022, 5, 30)), '2021/2022'),
            ((datetime(2022, 8, 5), datetime(2023, 6, 20)), '2022/2023')
        ],
        'Bundesliga': [
            ((datetime(2017, 8, 10), datetime(2018, 5, 27)), '2017/2018'),
            ((datetime(2018, 8, 5), datetime(2019, 5, 27)), '2018/2019'),
            ((datetime(2019, 8, 3), datetime(2020, 7, 30)), '2019/2020'),
            ((datetime(2020, 9, 5), datetime(2021, 5, 30)), '2020/2021'),
            ((datetime(2021, 8, 5), datetime(2022, 5, 30)), '2021/2022'),
            ((datetime(2022, 8, 5), datetime(2023, 5, 30)), '2022/2023')
        ],
        'Serie A': [
            ((datetime(2017, 8, 10), datetime(2018, 5, 27)), '2017/2018'),
            ((datetime(2018, 8, 5), datetime(2019, 5, 30)), '2018/2019'),
            ((datetime(2019, 8, 15), datetime(2020, 8, 5)), '2019/2020'),
            ((datetime(2020, 9, 5), datetime(2021, 5, 30)), '2020/2021'),
            ((datetime(2021, 8, 5), datetime(2022, 5, 30)), '2021/2022'),
            ((datetime(2022, 8, 5), datetime(2023, 6, 10)), '2022/2023')
        ],
        'La Liga': [
            ((datetime(2017, 8, 10), datetime(2018, 5, 27)), '2017/2018'),
            ((datetime(2018, 8, 5), datetime(2019, 5, 27)), '2018/2019'),
            ((datetime(2019, 8, 3), datetime(2020, 7, 30)), '2019/2020'),
            ((datetime(2020, 9, 5), datetime(2021, 5, 30)), '2020/2021'),
            ((datetime(2021, 8, 5), datetime(2022, 5, 30)), '2021/2022'),
            ((datetime(2022, 8, 5), datetime(2023, 6, 10)), '2022/2023')
        ],
        'US Major League Soccer': [
            ((datetime(2017, 2, 25), datetime(2017, 11, 10)), '2017'),
            ((datetime(2018, 2, 27), datetime(2018, 12, 10)), '2018'),
            ((datetime(2019, 2, 25), datetime(2019, 12, 15)), '2019'),
            ((datetime(2020, 2, 25), datetime(2020, 12, 20)), '2020'),
            ((datetime(2021, 4, 10), datetime(2021, 12, 5)), '2021'),
            ((datetime(2022, 2, 20), datetime(2022, 12, 5)), '2022')
        ],
    }
    
    # Iterar sobre los rangos de fechas para encontrar la temporada correcta
    for (start_date, end_date), season in seasons_dict.get(competition, []):
        if start_date <= date <= end_date:
            return season
    return None 

# Aplicar la función al dataframe
df_fbref['season'] = df_fbref.apply(lambda row: determine_fbref_season(row['datetime'], row['competition']), axis=1)

In [50]:
def determine_season(year, competition):
    # Diccionarios con los mapeos de año a temporada
    seasons_dict = {
        'Liga Profesional Argentina': {2017: '2017/2018', 2018: '2018/2019', 2019: '2019/2020', 2020: '2020', 2021: '2021', 2022: '2022'},
        'Brazilian Serie A': {2017: '2017', 2018: '2018', 2019: '2019', 2020: '2020', 2021: '2021', 2022: '2022'},
        'Premier League': {2017: '2017/2018', 2018: '2018/2019', 2019: '2019/2020', 2020: '2020/2021', 2021: '2021/2022', 2022: '2022/2023'},
        'Ligue 1': {2017: '2017/2018', 2018: '2018/2019', 2019: '2019/2020', 2020: '2020/2021', 2021: '2021/2022', 2022: '2022/2023'},
        'Bundesliga': {2017: '2017/2018', 2018: '2018/2019', 2019: '2019/2020', 2020: '2020/2021', 2021: '2021/2022', 2022: '2022/2023'},
        'Serie A': {2017: '2017/2018', 2018: '2018/2019', 2019: '2019/2020', 2020: '2020/2021', 2021: '2021/2022', 2022: '2022/2023'},
        'La Liga': {2017: '2017/2018', 2018: '2018/2019', 2019: '2019/2020', 2020: '2020/2021', 2021: '2021/2022', 2022: '2022/2023'},
        'US Major League Soccer': {2017: '2017', 2018: '2018', 2019: '2019', 2020: '2020', 2021: '2021', 2022: '2022'}
    }
    
    # Comprobamos si la competición y el año están en el diccionario
    if competition in seasons_dict and year in seasons_dict[competition]:
        return seasons_dict[competition][year]
    else:
        return None

df_xvalue['season'] = df_xvalue.apply(lambda row: determine_season(row['year'], row['competition']), axis=1)

In [51]:
#Hacer merge con year categorico
merged_df = pd.merge(df_fbref, df_xvalue, how="inner", on=['season', 'team', 'opponent', 'competition', 'goals_scored', 'goals_conceded'])

In [52]:
merged_df = merged_df.sort_values(by='datetime')

In [55]:
merged_df.shape

(23735, 43)

In [54]:
merged_df = merged_df.drop_duplicates(subset=['datetime', 'team', 'opponent', 'goals_scored', 'goals_conceded'])

In [16]:
nan_counts = merged_df.isnull().sum()
print(nan_counts)
nan_percentage = (merged_df.isnull().sum() / len(merged_df)) * 100
print(nan_percentage)


datetime                                   0
competition                                0
goals_scored                               0
goals_conceded                             0
opponent                                   0
possession                                 2
formation                                  0
shots                                      0
shots_on_target                            0
penaltys_made                              0
penaltys_attempted                         0
season                                     0
team                                       0
penalty_shootout                           0
side                                       0
year                                       0
expected_goals                             0
expected_goals_in_open_play                0
expected_goals_from_counters               0
expected_goals_from_set_pieces             0
shot_accuracy                              0
expected_goals_conceded                    0
expected_g

In [56]:
#Borrar x_goals_against, x_goals_x, year
cols_to_drop = ['year']
merged_df.drop(columns=cols_to_drop, inplace=True)

In [31]:
print(merged_df.shape)

(23735, 42)


In [18]:
# def define_strategy(row):
#     # Posesión: Alta posesión y alta cantidad de pases
#     if row['possession'] > 55 and row['total_passes'] > 450:
#         return "Posesion"

#     # Contraataque: Baja posesión pero alta cantidad de tiros o alto valor en goles esperados de contraataque
#     elif row['possession'] < 45 and (row['shots'] > 10 or row['shots_on_target'] > 5 or row['expected_goals_from_counters'] > 0.5):
#         return "Contraataque"

#     # Presión Alta: Bajo valor en ppda
#     elif row['ppda'] < 10:
#         return "Presion Alta"

#     # Juego por Banda o Centros: Alto número de crosses y actividad en touches_opponent_box
#     elif row['crosses'] > 20 and row['touches_opponent_box'] > 15:
#         return "Juego por Banda"

#     # Defensivo: Baja cantidad de tiros concedidos, alto defensive_event_height y bajo field_tilt
#     elif row['total_shots_conceded'] < 8 and row['defensive_event_height'] > 20 :
#         return "Defensivo"

#     # Equilibrado: Estrategia por defecto
#     else:
#         return "Equilibrado"
    
#merged_df['strategy'] = merged_df.apply(define_strategy, axis=1)

In [57]:
merged_df['formation'] = merged_df['formation'].replace({
    4321: 433,
    541: 532,
    3511:352,
    4312: 41212,
    451: 4141
})

In [58]:
def formation_based_strategy(row):
    formation = row['formation']
    
    if formation == 532:
        return "Contraataque"
    elif formation in [352, 343]:
        return "Presión Alta"
    elif formation in [433, 4231, 4141]:
        return "Posesión"
    elif formation in [442, 41212]:
        return "Juego Directo"
    else:
        return "Indefinido"  # Para cualquier caso que no coincida con las formaciones anteriores

merged_df['strategy'] = merged_df.apply(formation_based_strategy, axis=1)

In [62]:
def define_tactic(row):
    possession_threshold_4231 = 53  
    possession_threshold_433 = 50  
    crosses_threshold_442 = 18 
    ppda_threshold_352 = 12  

    if row['formation'] == 4231:
        if row['possession'] >= possession_threshold_4231:
            return "4231 - Posesion"
        else:
            return "4231 - Contraataque"

    elif row['formation'] == 433:
        if row['possession'] >= possession_threshold_433:
            return "433 - Posesion"
        else:
            return "433 - Juego Directo"

    elif row['formation'] == 352:
        if row['ppda'] <= ppda_threshold_352:
            return "352 - Presion Alta"
        else:
            return "352 - Posesion"

    elif row['formation'] == 343:
        return "343 - Presion Alta"

    elif row['formation'] == 442:
        if row['crosses'] >= crosses_threshold_442:
            return "442 - Juego Directo"
        else:
            return "442 - Posesion"

    elif row['formation'] == 4141:
        return "4141 - Posesion"

    elif row['formation'] == 41212:
        return "41212 - Juego Directo"

    elif row['formation'] == 532:
        return "532 - Contraataque"

    else:
        return "Indefinido"

merged_df['tactic'] = merged_df.apply(define_tactic, axis=1)


In [23]:
formation_counts = merged_df['formation'].value_counts()
print(formation_counts)

formation
4231     7665
433      3930
442      3382
352      2561
343      2196
4141     1439
41212     685
4312      541
532       515
541       319
3511      213
4321      160
451       129
Name: count, dtype: int64


In [34]:
formation_counts = merged_df['formation'].value_counts()
print(formation_counts)

formation
4231     7665
433      4090
442      3382
352      2774
343      2196
4141     1568
41212    1226
532       834
Name: count, dtype: int64


In [60]:
strategy_counts = merged_df['strategy'].value_counts()
print(strategy_counts)

strategy
Posesión         13323
Presión Alta      4970
Juego Directo     4608
Contraataque       834
Name: count, dtype: int64


In [63]:
tactic_counts = merged_df['tactic'].value_counts()
print(tactic_counts)

tactic
4231 - Contraataque      4083
4231 - Posesion          3582
433 - Posesion           2707
343 - Presion Alta       2196
442 - Posesion           1755
442 - Juego Directo      1627
4141 - Posesion          1568
352 - Presion Alta       1428
433 - Juego Directo      1383
352 - Posesion           1346
41212 - Juego Directo    1226
532 - Contraataque        834
Name: count, dtype: int64


In [66]:
from sklearn.preprocessing import MinMaxScaler

In [67]:
columns_to_exclude = ['formation', 'year']
cols_to_normalize = [col for col in merged_df.columns if (merged_df[col].dtype in ['int64', 'float64']) and (col not in columns_to_exclude)]
scaler = MinMaxScaler()

# Normalizar y agregar el sufijo "_norm"
for col in cols_to_normalize:
    merged_df[col + '_norm'] = scaler.fit_transform(merged_df[[col]])

In [68]:
print(merged_df.columns)

Index(['datetime', 'competition', 'goals_scored', 'goals_conceded', 'opponent',
       'possession', 'formation', 'shots', 'shots_on_target', 'penaltys_made',
       'penaltys_attempted', 'season', 'team', 'penalty_shootout', 'side',
       'expected_goals', 'expected_goals_in_open_play',
       'expected_goals_from_counters', 'expected_goals_from_set_pieces',
       'shot_accuracy', 'expected_goals_conceded',
       'expected_goals_conceded_in_open_play',
       'expected_goals_conceded_from_counters',
       'expected_goals_conceded_from_set_pieces', 'total_shots_conceded',
       'shots_on_target_conceded', 'defensive_event_height', 'field_tilt',
       'expected_threat_from_passes', 'total_passes', 'dribbles',
       'touches_opponent_box', 'through_balls', 'crosses', 'one_twos', 'ppda',
       'gpi', 'total_fouls_won', 'total_fouls_conceded', 'yellow_cards',
       'red_cards', 'expected_threat', 'strategy', 'tactic',
       'goals_scored_norm', 'goals_conceded_norm', 'possession_

In [69]:
merged_df.shape

(23735, 79)

In [70]:
cols_normalized = [col + '_norm' for col in cols_to_normalize]
cols_to_keep = ['datetime', 'team', 'opponent', 'tactic', 'formation', 'strategy'] + cols_normalized
df_norm = merged_df[cols_to_keep]

In [71]:
print(df_norm.columns)

Index(['datetime', 'team', 'opponent', 'tactic', 'formation', 'strategy',
       'goals_scored_norm', 'goals_conceded_norm', 'possession_norm',
       'shots_norm', 'shots_on_target_norm', 'penaltys_made_norm',
       'penaltys_attempted_norm', 'penalty_shootout_norm',
       'expected_goals_norm', 'expected_goals_in_open_play_norm',
       'expected_goals_from_counters_norm',
       'expected_goals_from_set_pieces_norm', 'shot_accuracy_norm',
       'expected_goals_conceded_norm',
       'expected_goals_conceded_in_open_play_norm',
       'expected_goals_conceded_from_counters_norm',
       'expected_goals_conceded_from_set_pieces_norm',
       'total_shots_conceded_norm', 'shots_on_target_conceded_norm',
       'defensive_event_height_norm', 'field_tilt_norm',
       'expected_threat_from_passes_norm', 'total_passes_norm',
       'dribbles_norm', 'touches_opponent_box_norm', 'through_balls_norm',
       'crosses_norm', 'one_twos_norm', 'ppda_norm', 'gpi_norm',
       'total_fouls_won

In [72]:
df_norm.shape

(23735, 41)

In [73]:
exclude_columns = ['formation', 'strategy']
stats_columns = [col for col in df_norm.columns if df_norm[col].dtype in ['int64', 'float64'] and col not in exclude_columns]
relevant_columns = ['datetime', 'team', 'opponent', 'tactic'] + stats_columns
filtered_data = df_norm[relevant_columns]

In [74]:
filtered_data['datetime'] = pd.to_datetime(filtered_data['datetime'])
sorted_data = filtered_data.sort_values(by=['team', 'datetime'])

C:\Users\GEEK STORE\AppData\Local\Temp\ipykernel_9556\1900454804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['datetime'] = pd.to_datetime(filtered_data['datetime'])


In [75]:
# Funcion para obtener las estadísticas de los 5 partidos anteriores
def get_previous_matches_stats(row, data, n=5):
    # Filtrar partidos anteriores del oponente
    previous_matches = data[(data['team'] == row['opponent']) & (data['datetime'] < row['datetime'])]
    
    # Ordenar por fecha y obtener los últimos n partidos
    previous_matches = previous_matches.sort_values(by='datetime', ascending=False).head(n)
    
    # Crear un diccionario para almacenar las estadísticas
    stats = {}
    for i in range(1, n+1):
        match = previous_matches.iloc[i-1] if i <= len(previous_matches) else pd.Series()
        for col in stats_columns:
            stats[f"{col}_{i}"] = match.get(col, None)

    return pd.Series(stats)

In [76]:
# Aplicar la función a cada fila
stats_data = sorted_data.apply(lambda row: get_previous_matches_stats(row, sorted_data), axis=1)
final_data = pd.concat([sorted_data.reset_index(drop=True), stats_data.reset_index(drop=True)], axis=1)

In [77]:
final_data = final_data.sort_values(by='datetime')

In [78]:
final_data.shape

(23735, 214)

In [79]:
final_data.to_csv('./final_data/rolled_data.csv')